In [11]:
import nbimporter
import numpy as np
import random
import time

In [12]:
import chromosome as Chromosome

In [10]:
chro2 = chr.Chromosome(np.arange(3))
chro2.get_positions()

array([0, 1, 2])

# class EvolManager

In [ ]:
class EvolManager:
    def __init__(self, n, pop, generations, matingPoolSize=80, offspring=80, noveltyAddRate=0.2):
        self.__n = n
        self.__population = pop
        self.__generations = generations
        self.__currentGeneration = 0
        self.__cromArray = []
        self.__solutions = []
        self.__solGeneration = []
        self.__matingPool = []
        self.__sortedCromArr = []
        self.__matingPoolSize = matingPoolSize
        self.__offspring = []
        self.__offspringSize = offspring
        self.__bigArr = []
        self.__novelArchive = []
        self.__noveltyAddRate = noveltyAddRate
        self.__solutionTimes = []
        
        self.populate()

    def populate(self):
        protoCro = Chromosome(list(range(self.__n)))

        for i in range(self.__population):
            self.__cromArray.append(Chromosome(protoCro.randomCrom()))

        for i in range(len(self.__cromArray)):
            new_sol = True
            if self.__cromArray[i].fitness() == 1.00:
                for j in range(len(self.__solutions)):
                    if self.__cromArray[i].getPositions() == self.__solutions[j].getPositions():
                        new_sol = False
                        break
                if new_sol:
                    self.__solutions.append(self.__cromArray[i])
                    self.__solGeneration.append(self.__currentGeneration)

        print("It's populated over here!")

    def getCromArray(self):
        return self.__cromArray

    def getSolutions(self):
        return self.__solutions

    def getSolutionTimes(self):
        return self.__solutionTimes

    def getBestCromFitness(self):
        ordered_popul = sorted(self.__cromArray, key=lambda crom: -crom.fitness())
        return ordered_popul[0].fitness()

    def getBestCromMutualThreats(self):
        ordered_popul = sorted(self.__cromArray, key=lambda crom: -crom.fitness())
        return ordered_popul[0].mutualThreats()

    def getMatingPool(self):
        return self.__matingPool

    
    def getOffspring(self):
        return self.__offspring
    

    def showChromosomes(self):
        print("cromArray members: ")
        for i in range(len(self.__cromArray)):
            print(f"{self.__cromArray[i].getPositions()} f: {self.__cromArray[i].fitness}")

            if self.__cromArray[i].fitness == 1.00:
                new_sol = True
                for j in range(len(self.__solutions)):
                    if self.__cromArray[i].getPositions() == self.__solutions[j].getPositions():
                        new_sol = False
                        break
                if new_sol:
                    self.__solutions[len(self.__solutions)] = self.__cromArray[i]
                    self.__solGeneration[len(self.__solutions) - 1] = self.__currentGeneration

    def showSolutions(self):
        print("Solutions: ")
        for i in range(len(self.__solutions)):
            print(f"@solutions[{i}]: {self.__solutions[i].getPositions()}. Generation: {self.__solGeneration[i]}")

        if len(self.__solutions) > 2:
            if self.__solutions[len(self.__solutions) - 1].getPositions() == self.__solutions[len(self.__solutions) - 2].getPositions():
                print("last solution repeated")

    def makeMatingPool(self):
        self.__matingPool.clear()

        # Sort chromosomes by fitness in descending order
        self.__sortedCromArr = sorted(self.__cromArray, key=lambda crom: -crom.fitness())

        c = 0
        while len(self.__matingPool) < self.__matingPoolSize:
            boolSignal = False
            d = c % self.__population

            if self.__n <= 2 or (c > self.__population and len(self.__matingPool) == 0):
                self.__matingPool = self.__sortedCromArr[:self.__matingPoolSize]

            # Chromosomes have a probability of entering the mating pool proportional to their fitness.
            # Chromosomes with fitness = 0 cannot enter the mating pool but it is extremely unlikely.
            elif random.random() < (self.__sortedCromArr[d].fitness / 3):
                self.__matingPool.append(self.__sortedCromArr[d])

            # If a solution is found, it is stored in the solutions array and if that solution was previously found,
            # it is removed from the mating pool
            if self.__sortedCromArr[d].fitness == 1.000000:
                if len(self.__solutions) == 0:
                    self.__solutions.append(self.__sortedCromArr[d])
                    self.__solGeneration.append(self.__currentGeneration)
                    ending = time.perf_counter()
                    lapse = ending - self.__starting
                    self.__solutionTimes.append(lapse)
                    print("Time for first solution:")
                    print(lapse)
                    self.__sortedCromArr.pop(d)
                    self.__sortedCromArr.append(Chromosome.new(self.__sortedCromArr[d - 1].randomCrom))
                    continue
                # If the solution was already found, it is removed from the mating pool
                for i in range(len(self.__solutions)):
                    if self.__solutions[i].getPositions() == self.__sortedCromArr[d].getPositions():
                        self.__matingPool.append(self.__sortedCromArr[d].makeChild())
                        self.__matingPool.remove(self.__sortedCromArr[d])
                        self.__cromArray.remove(self.__sortedCromArr[d])
                        self.__cromArray.append(Chromosome.new(self.__sortedCromArr[d - 1].randomCrom))
                        break
            c += 1


    def matingSeason(self):
        for i in range(self.__offspringSize):
            self.__offspring[i] = self.__matingPool[i % self.__matingPoolSize].makeChild()

    def replacement(self):
        self.__bigArr.clear()
        self.__bigArr = self.__cromArray + self.__offspring
        self.__bigArr = sorted(self.__bigArr, key=lambda crom: -crom.fitness())
        self.__cromArray = self.__bigArr[:self.__population]

    def greedyNextGeneration(self):
        print(f"@currentGeneration = {self.__currentGeneration}")
        self.makeMatingPool()
        self.matingSeason()
        self.replacement()

    def greedyEvolve(self):
        self.__currentGeneration = 0
        self.__cromArray.clear()
        self.__solutions.clear()
        self.__solGeneration.clear()
        self.__matingPool.clear()
        self.__sortedCromArr.clear()
        self.__offspring.clear()
        self.__bigArr.clear()
        self.__novelArchive.clear()
        self.__solutionTimes.clear()
        self.populate()
        self.__starting = time.perf_counter()

        while self.__currentGeneration < self.__generations:
            self.greedyNextGeneration()
            self.__currentGeneration += 1

        print(f"@currentGeneration = {self.__currentGeneration}")
        print(f"population.size = {self.__population}")
        self.showChromosomes()
        self.showSolutions()

    def getCurrentGeneration(self):
        return self.__currentGeneration


    def shortestDist(crom0, arr, index = -1):
        minDistance = 100
        for i in range(len(arr)):
            if i == index:
                continue
            if crom0.behavDist(arr[i]) < minDistance:
                minDistance = crom0.behavDist(arr[i])
                if minDistance == 0:
                    return minDistance
        return minDistance

    # Method that creates a mating pool by sorting the chromosomes in @cromArray by their novelty,
    # and adding them to the @matingPool with a probability of 0.9, unless the chromosome is already
    # present in the @novelArchive
    def noveltyMakeMatingPool(self):
        self.matingPool.clear()

        # sort chromosomes by their novelty
        if len(self.novelArchive) > 0:
            self.sortedCromArr = sorted(self.cromArray, key=lambda crom: -shortestDist(crom, self.novelArchive))
        else:
            self.sortedCromArr = self.cromArray

        c = 0 

        while len(self.matingPool) < self.matingPoolSize:
            boolSignal = False
            d = c % self.population

            # in case the population is less than or equal to 2 or if the mating pool is still empty after the population size
            if (self.n <= 2) or (c > self.population and len(self.matingPool) == 0):
                self.matingPool = self.sortedCromArr[:self.matingPoolSize]
        
            # add chromosome to mating pool with probability of 0.9
            elif (random.random() < 0.9):
                self.matingPool.append(self.sortedCromArr[d])
                if (not any(c.getPositions == self.sortedCromArr[d].getPositions for c in self.novelArchive)):
                    self.novelArchive.append(self.sortedCromArr[d])
                
            # Store new solutions, this does not affect novelty evolution
            if self.sortedCromArr[d].fitness() == 1.000000:
                if len(self.solutions) == 0:
                    self.solutions.append(self.sortedCromArr[d])
                    self.solGeneration.append(self.currentGeneration)
                    ending = Process.clock_gettime(Process.CLOCK_MONOTONIC)
                    lapse = ending - self.starting
                    self.solutionTimes.append(lapse)
                    print(f"Time for first solution: {lapse}")
                else:
                    boolSignal = False
                    for i in range(len(self.solutions)):
                        if self.solutions[i].getPositions == self.sortedCromArr[d].getPositions:
                            boolSignal = True
                            break
                    if boolSignal:
                        # not storing the solution
                        pass
                    else:
                        self.solutions.append(self.sortedCromArr[d])
                        self.solGeneration.append(self.currentGeneration)
                        ending = time.perf_counter()
                        lapse = ending - self.starting
                        self.solutionTimes.append(lapse)
                        print(f"Time for solution: {lapse}")
            c += 1


    def noveltyArchiveAdd(self, crom):
        """
        If the configuration of the received chromosome is not in @novelArchive, it is stored there.
        """
        for i in range(len(self.__novelArchive)):
            if crom.getPositions == self.__novelArchive[i].getPositions:
                return
        self.__novelArchive.append(crom)

    def noveltyReplacement(self):
        """
        Takes the new individuals obtained from reproduction (@offspring) 
        and joins them with the entire population. 
        Then selects the ones with the highest novelty to remain in the 
        population. The array where the population is located is @cromArray
        """
        self.__bigArr.clear()
        self.__bigArr = self.__cromArray + self.__offspring
        self.__bigArr.sort(key=lambda crom: -self.shortestDist(crom, self.__novelArchive))
        self.__cromArray = self.__bigArr[:self.__population]

    def noveltyNextGeneration(self):
        print("@currentGeneration = {}".format(self.__currentGeneration))
        self.noveltyMakeMatingPool()
        self.matingSeason()
        self.noveltyReplacement()
        for i in range(int(self.__population * self.__noveltyAddRate)):
            self.noveltyArchiveAdd(self.__cromArray[i])

    def showArrayOfCrom(self, arr):
        """
        Print positions of chromosomes
        """
        for i in range(len(arr)):
            print(arr[i].getPositions)


    def showChromosomesWithB(self):
        print("cromArray members: ")
        for i in range(len(self.__cromArray)):
            print(self.__cromArray[i].getPositions(), "b: ", self.__cromArray[i].getBehavior())

    def showChromosomesWithSDist(self):
        print("cromArray members: ")
        for i in range(len(self.__novelArchive)):
            print(self.__novelArchive[i].getPositions(), "sDist: ", self.shortestDist(self.__novelArchive[i], self.__novelArchive, i))
        print("@novelArchive.size = ", len(self.__novelArchive))

    def curiousEvolve(self):
        """Evolution algorithm that rewards novelty of chromosomes instead of how close they are to being a valid solution."""
        self._currentGeneration = 0
        self._cromArray.clear()
        self._solutions.clear()
        self._solGeneration.clear()
        
        self._matingPool.clear()
        self._sortedCromArr.clear()
        self._offspring.clear()
        self._bigArr.clear()
        self._novelArchive.clear()
        self._solutionTimes.clear()
        self.populate()
        self._starting = Process.clock_gettime(Process.CLOCK_MONOTONIC)

        while self._currentGeneration < self._generations:
            self.noveltyNextGeneration()
            self._currentGeneration += 1
            
        print("currentGeneration = ", self._currentGeneration)
        print("population.size = ", self._population)
        self.showChromosomes()
        self.showSolutions()


    def multiObjectiveParetoMakeMatingPool(self):
        self._matingPool = []
        ordByFitness = []
        ordByNovelty = []
        ordByFitness = sorted(self._cromArray, key = lambda crom : -crom.fitness(), reverse=True)

        if len(self._novelArchive) > 0:
            #Sorts chromosomes from highest to lowest novelty 
            ordByNovelty = sorted(self._cromArray, key = lambda crom : -self.shortestDist(crom, self._novelArchive), reverse=True)
        else:
            ordByNovelty = self._cromArray

        for c in range(5):
            self.noveltyArchiveAdd(ordByNovelty[c])
        while ( len(self._matingPool) < self._matingPoolSize ):
            ordByFitness = sorted(self._cromArray, key = lambda crom : -crom.fitness(), reverse=True)
            ordByNovelty = sorted(self._cromArray, key = lambda crom : -self.shortestDist(crom, self._novelArchive), reverse=True)
            for c in range(3):
                self.noveltyArchiveAdd(ordByNovelty[c])
            ind0 = 0
            while (ind0 < len(ordByFitness) and len(self._matingPool) < self._matingPoolSize):
                pareto = True
                newMate = True
                for ind1 in range(ind0):
                    if( ordByNovelty.index(ordByFitness[ind1]) <   
                            ordByNovelty.index(ordByFitness[ind0]) ):
                        pareto = False
                        break

                if ( pareto and newMate ):
                    self._matingPool.append(ordByFitness[ind0])
                    self._cromArray.remove(ordByFitness[ind0])
                    self._cromArray.append(Chromosome.randomCrom())

                #ind0 += 1  #possible error
                boolSignal = False
                if (ordByFitness[ind0].fitness == 1.000000):
                    if len(self._solutions) == 0:
                        self._solutions.append(ordByFitness[ind0])
                        self._solGeneration.append(self._currentGeneration)
                        ending = Process.clock_gettime(Process.CLOCK_MONOTONIC)
                        lapse = ending - self._starting
                        self._solutionTimes.append(lapse)
                        print("Time for solution:")
                        print(lapse)

                        # Initially, the idea was to remove the solutions by allowing a mutation 
                        # of the solution to enter the mating pool. But since novelty values the 
                        # novelty of a chromosome, this is not really necessary.


                        @cromArray.remove(ordByFitness[ind0])
                        @cromArray.append( Chromosome( @cromArray[0].randomCrom ) )
                        continue
                            
                    for i in range(len(@solutions)):
                        if ( @solutions[i].getPositions == ordByFitness[ind0].getPositions):
                            #@matingPool.append(ordByFitness[ind0].makeChild)
                            #@matingPool.remove(ordByFitness[ind0])
                            
                            @cromArray.remove(ordByFitness[ind0])
                            @cromArray.append(Chromosome(@cromArray[0].randomCrom))
                            boolSignal = True
                            break            
                    
                    if boolSignal:
                        pass
                        #continue
                    else:
                        @solutions.append(ordByFitness[ind0])
                        @solGeneration.append(@currentGeneration)
                        
                        ending = Process.clock_gettime(Process.CLOCK_MONOTONIC)
                        lapse = ending - @starting
                        @solutionTimes.append(lapse.dup())
                        print("Time for solution:")
                        print(lapse)
                    
                ind0 += 1
                # end of while loop for selecting chromosomes on the Pareto frontier


    def multiObjectiveReplacement(self):
        self.bigArr.clear()
        self.bigArr = self.__offspring + self.__cromArray
        closestToPareto = sorted(self.bigArr, key=lambda crom: -self.shortestDist(crom, self.__matingPool))
        self.__cromArray = closestToPareto[:self.__population]


    
    def multiObjectiveNextGeneration(self):
        print("@currentGeneration = ", self.__currentGeneration)
        self.multiObjectiveParetoMakeMatingPool()
        self.matingSeason() #Same method as for greedyEvolve
        self.multiObjectiveReplacement()

        for i in range(int(self.__population * 0.8)): #Parameter for curiousEvolve noveltyAddRate
            self.noveltyArchiveAdd(self.__cromArray[i])


    def __multiObjectiveEvolve(self):
        self.__currentGeneration = 0
        self.__cromArray.clear()
        self.__solutions.clear()
        self.__solGeneration.clear()

        self.__matingPool.clear()
        self.__sortedCromArr.clear()
        self.__offspring.clear()
        self.__bigArr.clear()
        self.__novelArchive.clear()
        self.__solutionTimes.clear()

        self.populate()
        self.__starting = Process.clock_gettime(Process.CLOCK_MONOTONIC)

        while self.__currentGeneration < self.__generations:
            self.__multiObjectiveNextGeneration()
            self.__currentGeneration += 1

        print(f"currentGeneration = {self.__currentGeneration}")
        print(f"population.size = {self.__population}")
        self.showChromosomes()
        self.showSolutions()



# Testing

## Testing value pass in the makeMatingPool() method

## Check noveltyMakeMatingPool

## Replace 
* @
* check att names everywhere
* Process with corresponding time.time()